# Container: torchfort.sif

Runnin on the SDumont

In [2]:
! singularity --version

singularity-ce version 4.2.1-1


<https://github.com/sylabs/singularity/releases/tag/v4.2.1>

Sep 13, 2024

In [3]:
%%bash
singularity shell --nv $HOME/containers/torchfort.sif
nvidia-smi
exit

Wed Jun  4 16:14:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           On  |   00000000:3B:00.0 Off |                    0 |
| N/A   36C    P0             25W /  250W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Equivalences Docker (D) vs Singularity (S)

    --gpus=all → On Singularity, we use --nv to enable NVIDIA GPUs.
    -v /host:/container → On Singularity, we use --bind /host:/container.
    -p → Singularity doesn't manage ports directly like Docker, so it needs alternatives like external SSH redirection.
    --user 1000:1000 → Singularity inherits the system user, so there is no need to set a user ID.


| **Docker**                        | **Singularity**                          |
|------------------------------------|------------------------------------------|
| `docker run -d --name t01 image`  | `singularity instance start image.sif t01` |
| `docker exec -it t01 bash`        | `singularity exec instance://t01 bash` |
| `docker stop t01`                 | `singularity instance stop t01` |

- In Singularity, when stopping the instance, the changes are lost. You need to save first with the command: `singularity build new_container.sif torchfort_v4.sif`
- Singularity automatically mounts the host directories: `$HOME, $PWD, /tmp, /var/tmp, /proc, /sys, /dev, /etc/resolv.conf, /etc/passwd, /etc/group`
- In Singularity, containers run in the same network namespace as the host system, so access can be more direct than in containers like Docker.
- On Singularity, by default, images (.sif) are read-only. To be able to write to them, you can mount directories on the host using --bind, use a writable tmpfs overlay filesystem that allocates a small amount of RAM for this purpose with --writable-tmpfs, create an overlay (like a union filesystem) with an external file and using --overlay, and create a directory as a container using --sandbox.
- The Singularity container automatically mounts the instance's `$HOME` as the host's `$HOME`, so it ends up mixing files used by both the container and the host in `$HOME`. This behavior may not always be desirable, especially if the operating systems and software used in the container and the host are different. An example is the conda environment, which may exist in the host's `$HOME` and another environment in the container. In this case it would be better to use the `--no-home` flag and mount the host's `$HOME` in `mnt/`, e.g. with the `--bind $HOME:/mnt/x` flag.

Interactive:

    singularity shell --nv $HOME/containers/torchfort.sif

Background run:

In [4]:
! singularity instance start --nv $HOME/containers/torchfort.sif t01

INFO:    Instance stats will not be available - system configuration does not support cgroup management.
INFO:    instance started successfully


Other commands:

In [5]:
! singularity exec instance://t01 pwd

/prj/ampemi/eduardo.miranda2/radnn/torchfort


In [6]:
! singularity instance list

INSTANCE NAME    PID        IP    IMAGE
t01              3818429          /prj/ampemi/eduardo.miranda2/containers/torchfort.sif


In [7]:
! singularity instance stop t01

INFO:    Stopping t01 instance of /prj/ampemi/eduardo.miranda2/containers/torchfort.sif (PID=3818429)


In [1]:
TFT="~/containers/torchfort.sif"

In [4]:
! singularity exec --nv {TFT} mpirun --version

mpirun (Open MPI) 4.1.7a1

Report bugs to http://www.open-mpi.org/community/help/


In [7]:
! singularity exec --nv {TFT} dpkg -l | grep openmpi

In [1]:
! cat /etc/redhat-release

Red Hat Enterprise Linux release 8.8 (Ootpa)


In [2]:
! cat /etc/os-release

NAME="Red Hat Enterprise Linux"
VERSION="8.8 (Ootpa)"
ID="rhel"
ID_LIKE="fedora"
VERSION_ID="8.8"
PLATFORM_ID="platform:el8"
PRETTY_NAME="Red Hat Enterprise Linux 8.8 (Ootpa)"
ANSI_COLOR="0;31"
CPE_NAME="cpe:/o:redhat:enterprise_linux:8::baseos"
HOME_URL="https://www.redhat.com/"
DOCUMENTATION_URL="https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/8"
BUG_REPORT_URL="https://bugzilla.redhat.com/"

REDHAT_BUGZILLA_PRODUCT="Red Hat Enterprise Linux 8"
REDHAT_BUGZILLA_PRODUCT_VERSION=8.8
REDHAT_SUPPORT_PRODUCT="Red Hat Enterprise Linux"
REDHAT_SUPPORT_PRODUCT_VERSION="8.8"


In [3]:
! mpirun --version

/bin/bash: /prj/ampemi/eduardo.miranda2/.local/bin/mpirun: /scratch/ampemi/eduardo.miranda2/miniconda3/envs/tf1/bin/python: bad interpreter: No such file or directory
